In [115]:
import imaplib
import email
from email.header import decode_header
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

In [116]:
def get_emails(username, password):
  mail = imaplib.IMAP4_SSL("imap.gmail.com")
  mail.login(username, password)
  mail.select("inbox")

In [117]:
get_emails = "faganfoils@gmail.com", "rvgqnmmymrvaalex"

In [118]:
status, messages = mail.select('inbox')

In [119]:
status, messages = mail.search(None, 'ALL')

In [120]:
email_ids = messages[0].split()

In [121]:
train_data = pd.read_csv('foil_training_data.csv')

In [122]:
X = train_data['response_content']
y = train_data['response_category']

In [123]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [124]:
pipeline = make_pipeline(TfidfVectorizer(), LogisticRegression())

In [125]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression', LogisticRegression())])

In [126]:
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      denial       0.58      0.70      0.64        10
   extension       0.00      0.00      0.00         1
       other       0.44      0.64      0.52        11
     records       0.60      0.27      0.38        11

    accuracy                           0.52        33
   macro avg       0.41      0.40      0.38        33
weighted avg       0.52      0.52      0.49        33



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [127]:
def sort_email(email_content):
    return pipeline.predict([email_content])[0]

In [128]:
status, messages = mail.search(None, 'UNSEEN')
email_ids = messages[0].split()
all_emails = mail, email_ids

In [129]:
def extract_email_body(msg):
    if msg.is_multipart():
        for part in msg.walk():
            content_type = part.get_content_type()
            content_disposition = str(part.get("Content-Disposition"))
            if "attachment" not in content_disposition:
                if content_type == "text/plain":
                    return part.get_payload(decode=True).decode()
                elif content_type == "text/html":
                    return part.get_payload(decode=True).decode()
    else:
        return msg.get_payload(decode=True).decode()

In [130]:
results = []

for email_id in email_ids:
        status, msg_data = mail.fetch(email_id, "(RFC822)")

        for response_part in msg_data:
            if isinstance(response_part, tuple):
                msg = email.message_from_bytes(response_part[1])
                body = extract_email_body(msg)

                label = classify_email(body)

                results.append((body, label))

In [131]:
df = pd.DataFrame(results, columns=["email_content", "predicted_label"])
df

,email_content,predicted_label
0,Please see the attached documents regarding yo...,records
